In [1]:
import pandas as pd

In [4]:
h_results = pd.read_csv("section data/H_popvote_df_fin.csv")
h_results.head()

,Unnamed: 0,year,party,winner_party,seats,Other_seats,votes,total_votes,majorvote_pct
0,1,1948,R,D,171,17,21613079,49560252,46.60
1,2,1948,D,D,263,17,24762972,49560252,53.40
2,3,1950,R,R,199,16,20925177,44050445,50.29
3,4,1950,D,R,234,16,20681636,44050445,49.71
4,5,1952,R,D,221,19,28860092,61803866,48.82


In [5]:
inc_results = pd.read_csv("section data/inc_pop_vote_df.csv")
inc_results.head()

,Unnamed: 0,year,party,winner_party.x,seats,Other_seats.x,votes,total_votes.x,majorvote_pct,president_party,H_incumbent_party_winner,H_incumbent_party
0,1,1948,R,D,171,17,21613079,49560252,46.60,D,False,R
1,2,1948,D,D,263,17,24762972,49560252,53.40,D,False,R
2,3,1950,R,R,199,16,20925177,44050445,50.29,D,False,D
3,4,1950,D,R,234,16,20681636,44050445,49.71,D,False,D
4,5,1952,R,D,221,19,28860092,61803866,48.82,R,False,R


In [35]:
inc_party_seats = inc_results.loc[list(inc_results.groupby("year")["seats"].idxmax()),["year", "party"]]
inc_party_seats["year"] = inc_party_seats["year"] + 2
inc_party_seats.head()

,year,party
1,1950,D
3,1952,D
4,1954,R
7,1956,D
9,1958,D


In [6]:
GDP_data = pd.read_csv("section data/Copy of GDP_quarterly.csv")
GDP_data.head()

,Unnamed: 0,...1,date,GDPC1,quarter_yr,quarter_cycle,year,year_qt,GDP_growth_qt,GDP_growth_pct
0,1,1,1/1/47,2033.061,1,1,1947,1947_1,0.000,0.000000
1,2,2,4/1/47,2027.639,2,2,1947,1947_2,-5.422,-0.266691
2,3,3,7/1/47,2023.452,3,3,1947,1947_3,-4.187,-0.206496
3,4,4,10/1/47,2055.103,4,4,1947,1947_4,31.651,1.564208
4,5,5,1/1/48,2086.017,1,5,1948,1948_5,30.914,1.504256


In [44]:
gdp_q8 = GDP_data[GDP_data["year_qt"].str.contains("_8")][["year", "GDP_growth_pct"]]
gdp_q8.head()

,year,GDP_growth_pct
7,1948,0.111742
15,1950,1.914763
23,1952,3.284198
31,1954,1.956683
39,1956,1.645929


In [48]:
inc_results.groupby("year")["president_party"].max()

year
1948    D
1950    D
1952    R
1954    R
1956    R
1958    R
1960    D
1962    D
1964    D
1966    D
1968    R
1970    R
1972    R
1974    R
1976    D
1978    D
1980    R
1982    R
1984    R
1986    R
1988    R
1990    R
1992    D
1994    D
1996    D
1998    D
2000    R
2002    R
2004    R
2006    R
2008    D
2010    D
2012    D
2014    D
2016    R
2018    R
2020    D
Name: president_party, dtype: object

In [49]:
data_df = pd.merge(inc_party_seats[:-1], gdp_q8, on="year", how="left")

In [53]:
data_df = pd.merge(data_df, inc_results.groupby("year")["president_party"].max(), on="year", how="left")

In [55]:
data_df["pres_inc"] = data_df["president_party"] == data_df["party"]

In [64]:
data_df = pd.merge(data_df, inc_results, on=["year", "party"], how="left")[["year", "party", "GDP_growth_pct", "pres_inc", "majorvote_pct"]]

In [65]:
data_df["midterm_year"] = (data_df["year"] % 2 == 0) & (data_df["year"] % 4 != 0)

In [66]:
data_df.head()

,year,party,GDP_growth_pct,pres_inc,majorvote_pct,midterm_year
0,1950,D,1.914763,True,49.71,True
1,1952,D,3.284198,False,51.18,False
2,1954,R,1.956683,True,47.60,True
3,1956,D,1.645929,False,51.25,False
4,1958,D,2.339305,False,54.97,True


In [63]:
from sklearn.linear_model import LinearRegression

In [140]:
from sklearn.model_selection import cross_val_score

In [67]:
non_poll_lm = LinearRegression()

In [72]:
non_poll_lm.fit(data_df[["GDP_growth_pct", "pres_inc", "midterm_year"]], data_df["majorvote_pct"])

LinearRegression()

In [73]:
non_poll_lm.predict(data_df[["GDP_growth_pct", "pres_inc", "midterm_year"]])

array([49.31692711, 50.60988166, 49.26897855, 52.48374606, 50.80524423,
       53.86046204, 51.13089435, 52.03914627, 50.5686996 , 53.91650965,
       54.70671387, 52.45111066, 53.92491871, 51.56337565, 49.96960276,
       52.23325663, 53.43527323, 53.42746315, 52.86664836, 52.84225337,
       54.52225872, 51.19928057, 50.19678806, 53.17866304, 51.63270347,
       51.67999447, 51.32996042, 51.24673781, 50.53267773, 54.86744436,
       50.93296776, 54.23612327, 52.83723191, 51.67266408, 51.13151117,
       50.72188725])

In [81]:
non_poll_lm.score(data_df[["GDP_growth_pct", "pres_inc", "midterm_year"]], data_df["majorvote_pct"])

0.22656164635584275

In [150]:
cross_val_score(non_poll_lm, data_df[["GDP_growth_pct", "pres_inc", "midterm_year"]], data_df["majorvote_pct"], cv=5, scoring="r2")

array([ 0.06838848, -0.91650098, -0.53891921, -0.88922062, -0.36132784])

In [82]:
polls_df = pd.read_csv("section data/GenericPolls1942_2020.csv")

In [103]:
year_spread = polls_df.groupby("year")[["dem", "rep"]].mean()
year_spread = year_spread["dem"] - year_spread["rep"]
year_spread.head()

year
1942    10.043478
1943     6.000000
1944    -1.000000
1945     9.000000
1946    -2.318182
dtype: float64

In [109]:
data_df = pd.merge(data_df, pd.DataFrame(year_spread, columns=["dem_year_spread"]), on="year", how="left")

In [110]:
data_df["is_dem"] = ((data_df["party"] == "D") + 0.5) * 2 - 2

In [111]:
data_df["inc_year_spread"] = data_df["dem_year_spread"] * data_df["is_dem"]

In [87]:
polls_df[polls_df["days_until_election"] < 20].groupby("year")[["dem", "rep"]].mean()

,dem,rep
year,,
1942,41.500000,45.500000
1946,38.000000,51.000000
1950,48.750000,43.750000
1954,53.000000,39.500000
1956,53.000000,42.000000
1958,52.000000,38.000000
1960,56.000000,37.000000
1962,55.500000,41.000000
1964,62.000000,36.000000


In [112]:
day20_spread = polls_df[polls_df["days_until_election"] < 20].groupby("year")[["dem", "rep"]].mean()
day20_spread = day20_spread["dem"] - day20_spread["rep"]
day20_spread.head()

year
1942    -4.0
1946   -13.0
1950     5.0
1954    13.5
1956    11.0
dtype: float64

In [113]:
data_df = pd.merge(data_df, pd.DataFrame(day20_spread, columns=["dem_day20_spread"]), on="year", how="left")

In [114]:
data_df["inc_day20_spread"] = data_df["dem_day20_spread"] * data_df["is_dem"]

In [125]:
data_df.loc[data_df["year"] == 1952, ["dem_day20_spread", "inc_day20_spread"]] = -1.72
data_df.loc[data_df["year"] == 1972, ["dem_day20_spread", "inc_day20_spread"]] = 8

In [126]:
data_df

,year,party,GDP_growth_pct,pres_inc,majorvote_pct,midterm_year,is_dem,dem_year_spread,inc_year_spread,dem_day20_spread,inc_day20_spread
0,1950,D,1.914763,True,49.71,True,1.0,9.555556,9.555556,5.000000,5.000000
1,1952,D,3.284198,False,51.18,False,1.0,-1.722222,-1.722222,-1.720000,-1.720000
2,1954,R,1.956683,True,47.60,True,-1.0,9.266667,-9.266667,13.500000,-13.500000
3,1956,D,1.645929,False,51.25,False,1.0,10.588235,10.588235,11.000000,11.000000
4,1958,D,2.339305,False,54.97,True,1.0,19.071429,19.071429,14.000000,14.000000
5,1960,D,-1.283434,True,54.35,False,1.0,20.266667,20.266667,19.000000,19.000000
6,1962,D,0.328861,True,52.87,True,1.0,22.000000,22.000000,14.500000,14.500000
7,1964,D,0.308893,True,57.27,False,1.0,28.266667,28.266667,26.000000,26.000000
8,1966,D,0.820372,True,50.86,True,1.0,15.608696,15.608696,7.500000,7.500000
9,1968,D,0.393303,False,50.92,False,1.0,10.600000,10.600000,8.000000,8.000000


In [127]:
year_poll_lm = LinearRegression()

In [128]:
year_poll_lm.fit(data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_year_spread"]], data_df["majorvote_pct"])
year_poll_lm.score(data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_year_spread"]], data_df["majorvote_pct"])

0.6583109403386616

In [148]:
cross_val_score(year_poll_lm, data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_year_spread"]], data_df["majorvote_pct"], cv=5, scoring="r2")

array([ 0.27109986,  0.4135977 , -0.34677593,  0.45554969,  0.13450302])

In [131]:
day20_poll_lm = LinearRegression()

In [133]:
day20_poll_lm.fit(data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_day20_spread"]], data_df["majorvote_pct"])
day20_poll_lm.score(data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_day20_spread"]], data_df["majorvote_pct"])

0.6506891004147476

In [149]:
cross_val_score(day20_poll_lm, data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_day20_spread"]], data_df["majorvote_pct"], cv=5, scoring="r2")

array([ 0.07684978,  0.46371557, -0.46330369,  0.42532091,  0.06707062])

In [134]:
from sklearn.ensemble import RandomForestRegressor

In [135]:
year_poll_rf = RandomForestRegressor(n_estimators=1000, random_state=0)

In [136]:
year_poll_rf.fit(data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_year_spread"]], data_df["majorvote_pct"])
year_poll_rf.score(data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_year_spread"]], data_df["majorvote_pct"])

0.9104409609125385

In [138]:
day20_poll_rf = RandomForestRegressor(n_estimators=1000, random_state=0)

In [139]:
day20_poll_rf.fit(data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_day20_spread"]], data_df["majorvote_pct"])
day20_poll_rf.score(data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_day20_spread"]], data_df["majorvote_pct"])

0.9094872507112068

In [146]:
cross_val_score(year_poll_rf, data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_year_spread"]], data_df["majorvote_pct"], cv=5, scoring="r2")

array([ 0.7366654 ,  0.24694415, -1.57723812, -0.29715671, -0.19058441])

In [147]:
cross_val_score(day20_poll_rf, data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_day20_spread"]], data_df["majorvote_pct"], cv=5, scoring="r2")

array([ 0.70168238,  0.2709529 , -2.34824751,  0.00272023, -0.19069632])

In [152]:
pres_approval_df = pd.read_csv("section data/Other polling data/pres_approval_gallup_1941-2022.csv")
pres_approval_df.head()

,poll_startdate,poll_enddate,approve,disapprove,unsure_NA,president,year
0,8/1/22,8/23/22,44,53,0,Joe Biden,2022
1,7/5/22,7/26/22,38,59,3,Joe Biden,2022
2,6/1/22,6/20/22,41,57,3,Joe Biden,2022
3,5/2/22,5/22/22,41,54,5,Joe Biden,2022
4,4/1/22,4/19/22,41,56,3,Joe Biden,2022


In [162]:
pres_year_spread = pres_approval_df.groupby("year")[["approve", "disapprove"]].mean()
pres_year_spread = pres_year_spread["approve"] - pres_year_spread["disapprove"]

In [166]:
data_df = pd.merge(data_df, pd.DataFrame(pres_year_spread, columns=["pres_year_spread"]), on="year", how="left")

In [169]:
data_df["is_pres_inc"] = (data_df["pres_inc"] + 0.5) * 2 - 2

In [170]:
data_df["inc_pres_year_spread"] = data_df["pres_year_spread"] * data_df["is_pres_inc"]

In [171]:
year_pres_lm = LinearRegression()

In [172]:
year_pres_lm.fit(data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_pres_year_spread"]], data_df["majorvote_pct"])
year_pres_lm.score(data_df[["GDP_growth_pct", "pres_inc", "midterm_year", "inc_pres_year_spread"]], data_df["majorvote_pct"])

0.28447624365634894

In [193]:
pres_approval_df["poll_startdate"]
abc = pd.to_datetime(pres_approval_df["poll_startdate"])

In [ ]:
data_df.to_csv("jd_data.csv", index=False)